In [ ]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys; sys.path.insert(0, '../')
import pickle as pkl
import numpy as np
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet import forward

plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [ ]:
info = forward.get_info()
info['sfreq'] = 100
fwd = forward.create_forward_model(info=info)
fwd_free = forward.create_forward_model(info=info, fixed_ori=False)

# Load Models

In [ ]:
lstm_standard = util.load_net('models/LSTM Medium_1-1000points_standard-cosine-mse_0')
lstm_noise = util.load_net('models/LSTM Medium_1-1000points_noise-cosine-mse_0')

dense_standard = util.load_net('models/Dense Medium_1-1000points_standard-cosine-mse_0')
dense_noise = util.load_net('models/Dense Medium_1-1000points_noise-cosine-mse_0')

convdip_standard = util.load_net('models/ConvDip Medium_1-1000points_standard-cosine-mse_0')
convdip_noise = util.load_net('models/ConvDip Medium_1-1000points_noise-cosine-mse_0')

models = [lstm_standard, lstm_noise, dense_standard, dense_noise, convdip_standard, convdip_noise]
model_names = ['LSTM Standard', 'LSTM Noise', 'Dense Standard', 'Dense Noise', 'ConvDip Standard', 'ConvDip Noise']

# Test ability to process long sequences

In [ ]:
%load_ext autoreload
%autoreload 2

settings_eval = dict(duration_of_trial=100, method='standard', number_of_sources=10)
sim_test = Simulation(fwd, info, settings=settings_eval).simulate(2)
idx = 0
durs = [100, 50, 10, 1, 0.1, 0.02]
# durs = [np.random.uniform(0.01, 100) for _ in range(5)]
for net, name in zip(models, model_names):
    for dur in durs:
        sim_test.crop(tmax=dur)
        prediction = net.predict(sim_test)
        error = util.batch_nmse(sim_test.source_data[idx].data, prediction[idx].data)
        r = util.batch_corr(sim_test.source_data[idx].data, prediction[idx].data)
        title = f'{dur}: error: {error:.4}, r: {r:.2f}\n'
        print(f"{name}:")
        print(title)


# Plot single ground truth and predictions

In [ ]:
import seaborn as sns
%matplotlib qt
sns.reset_orig()

%load_ext autoreload
%autoreload 2

plot_params = dict(surface='white', hemi='both', verbose=0, 
    clim=dict(kind='percent', pos_lims=[20, 30, 100]))

settings_eval = dict(duration_of_trial=1, method='standard')
# settings_eval = dict(duration_of_trial=1, method='noise', exponent=3)

# Simulate new data
sim_test = Simulation(fwd, info, settings=settings_eval).simulate(2)
snr = sim_test.simulation_info['target_snr'].values[0]
# print(sim_test.simulation_info)
idx = 0
# Predict sources using the esinet models
predictions = [model.predict(sim_test) for model in models]

# # Predict sources with classical methods
# # eLORETA
# prediction_elor_data = util.wrap_mne_inverse(fwd, sim_test, snr=snr)[idx].data.astype(np.float32)
# prediction_elor = deepcopy(predictions[0][0])
# prediction_elor.data = prediction_elor_data / np.abs(np.max(prediction_elor_data))
# # MNE
# prediction_mne_data = util.wrap_mne_inverse(fwd, sim_test, method='MNE', snr=snr)[idx].data.astype(np.float32)
# prediction_mne = deepcopy(predictions[0][0])
# prediction_mne.data = prediction_mne_data / np.abs(np.max(prediction_mne_data))
# # Beamformer
# prediction_lcmv_data = util.wrap_mne_inverse(fwd_free, sim_test, method='beamformer', snr=snr)[idx].data.astype(np.float32)
# prediction_lcmv = deepcopy(predictions[0][0])
# prediction_lcmv.data = prediction_lcmv_data / np.abs(np.max(prediction_lcmv_data))

# Get predictions and names in order
# predictions.append([prediction_elor])
# predictions.append([prediction_mne])
# predictions.append([prediction_lcmv])

# model_names.append('eLORETA')
# model_names.append('MNE')
# model_names.append('Beamformer')

# Plot True Source
brain = sim_test.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, f'Ground Truth {sim_test.simulation_info.number_of_sources.values[0]} sources, snr={snr:.1f}', 'title')
# Plot True EEG
# evoked = sim_test.eeg_data[idx].average()
# evoked.plot()
# evoked.plot_topomap(title='Ground Truth')
# evoked = util.get_eeg_from_source(sim_test.source_data[idx], fwd, info, tmin=0.)
# evoked.plot_topomap(title='Ground Truth Noiseless')

model_selection = model_names#['net_lstm_standard', 'net_lstm_noise', 'eLORETA']
# Plot predicted sources
for model_name, prediction in zip(model_names, predictions):
    
    if not any([model_name.lower() in model_select.lower() for model_select in model_selection]):
        continue
    error = util.batch_nmse(sim_test.source_data[idx].data, prediction[idx].data)
    r = util.batch_corr(sim_test.source_data[idx].data, prediction[idx].data)
    
    brain = prediction[idx].plot(**plot_params)

    title = f'{model_name}, error: {error:.4}, r: {r}'
    print(title)
    brain.add_text(0.1, 0.9, title, 'title')
    # Plot predicted EEG
    # evoked_esi = util.get_eeg_from_source(prediction[idx], fwd, info, tmin=0.)
    # evoked_esi.plot_topomap(title=model_name)


## Create or Load Evaluation Set

In [192]:
n_samples = 100
duration_of_trial = (0.01, 10)
method = 'standard'
settings = dict(duration_of_trial=duration_of_trial, method=method)
sim_test = Simulation(fwd, info, verbose=False, settings=settings).simulate(n_samples=n_samples)
# if type(duration_of_trial) == tuple:
#     sim_test.save(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial[0]*100)}-{int(duration_of_trial[1]*100)}points_{method}.pkl')
# else:
#     sim_test.save(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial*100)}points_{method}.pkl')

# or Load
# with open(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial*100)}points_{method}.pkl', 'rb') as f:
#     sim_test = pkl.load(f)

Simulating data based on sparse patches.


100%|██████████| 100/100 [00:00<00:00, 444.43it/s]


source data shape:  (1284, 714) (1284, 160)


100%|██████████| 100/100 [00:06<00:00, 16.16it/s]


In [193]:
from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
from esinet.util import wrap_mne_inverse
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

%load_ext autoreload
%autoreload 2
model_names_tmp = deepcopy(model_names)
# Predict
print('predict esinets...')
predictions = [model.predict(sim_test) for model in models]

print('predict elor')
pred_elor = wrap_mne_inverse(fwd, sim_test, method='eLORETA')
model_names_tmp.append('eLORETA')
predictions.append(pred_elor)

print('predict MNE')
pred_mne = wrap_mne_inverse(fwd, sim_test, method='MNE')
model_names_tmp.append('MNE')
predictions.append(pred_mne)

print('predict LCMV')
pred_lcmv = wrap_mne_inverse(fwd_free, sim_test, method='beamformer')
model_names_tmp.append('LCMV')
predictions.append(pred_lcmv)

pos = util.unpack_fwd(fwd)[2]
argsorted_distance_matrix = np.argsort(cdist(pos, pos), axis=-1)

mean_localization_errors = []
aucs = []
nmses = []
mses = []
true_sources = np.concatenate([src.data for src in sim_test.source_data], axis=1).T
print(true_sources.shape)
# size = 500
# n_total = true_sources.shape[0]
# choice = np.random.choice(np.arange(n_total), size=size, replace=False)
# true_sources = true_sources[choice]
for prediction, model_name in tqdm(zip(predictions, model_names_tmp)):
    print('\n', model_name, ':\n')
    predicted_sources = np.concatenate([src.data for src in prediction], axis=1).T

    # predicted_sources = predicted_sources[choice]
    print(predicted_sources.shape)
    print('mle calculation....')
    mean_localization_error = [eval_mean_localization_error(true_source, predicted_source, pos, argsorted_distance_matrix=argsorted_distance_matrix) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    # auc = [eval_auc(true_source, predicted_source, pos, epsilon=0.25, n_redraw=25) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    auc = Parallel(n_jobs=-1, backend='loky') \
        (delayed(eval_auc)(true_source, predicted_source, pos, epsilon=0.25, n_redraw=25)
        for true_source, predicted_source in zip(true_sources, predicted_sources))
    nmse = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    mse = [eval_mse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    
    mean_localization_errors.append(mean_localization_error)
    aucs.append(auc)
    nmses.append(nmse)
    mses.append(mse)

aucs_far = [auc[:, 1] for auc in np.array(aucs)]
aucs_close = [auc[:, 0] for auc in np.array(aucs)]
aucs_combined = [(auc[:, 0]+auc[:, 1])/2 for auc in np.array(aucs)]
metrics = dict(
    mean_localization_errors=mean_localization_errors,
    aucs_far=aucs_far,
    aucs_close=aucs_close,
    aucs_combined=aucs_combined,
    nmses=nmses,
    mses=mses
)

with open(f'results\\metrics_{len(true_sources)}_{sim_test.source_data[0].data.shape[1]}points_{method}.pkl', 'wb') as f:
    pkl.dump([metrics, None], f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
predict esinets...
interpolating for convdip...


100it [00:37,  2.69it/s]


interpolating for convdip...


100it [00:37,  2.69it/s]


predict elor
predict MNE
predict LCMV
(50807, 1284)


...

0it [00:00, ?it/s]


 Beamformer :

(50807, 1284)
mle calculation....


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Temporal information

In [ ]:
from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

tmax = sim_test.source_data[0].times.max()
tmin = sim_test.source_data[0].times.min()
eval_tmin = tmax - 1
new_tmins = [0.0, 4, 16, 18, 18.25, 18.5, 18.75, eval_tmin]
sim_cropped = deepcopy(sim_test)
pos = util.unpack_fwd(fwd)[2]
argsorted_distance_matrix = np.argsort(cdist(pos, pos), axis=-1)

mean_localization_errors = {model_name: [] for model_name in model_names}
aucs = {model_name: [] for model_name in model_names}
nmses = {model_name: [] for model_name in model_names}
mses = {model_name: [] for model_name in model_names}
true_sources = np.stack([src.crop(tmin=eval_tmin, tmax=tmax, include_tmax=True).data for src in deepcopy(sim_cropped.source_data)], axis=0)
true_sources = util.collapse(true_sources)
for i, new_tmin in enumerate(new_tmins):
    sim_cropped = sim_cropped.crop(tmin=new_tmin, tmax=tmax, include_tmax=True)
    print(f'new tmin: {sim_cropped.source_data[0].times.min()}, tmax: {sim_cropped.source_data[0].times.max()}')
    for model, model_name in zip(models, model_names):
        print('\t', model_name)
        print(f'\tpredicting {len(sim_cropped.source_data[0].times)} time points')
        prediction = model.predict(sim_cropped)
        # eval_tmin = tmax-1
        predicted_sources = [pred.crop(tmin=eval_tmin, tmax=tmax, include_tmax=True).data for pred in prediction]
        print(f'\ttesting {predicted_sources[0].shape[1]} time points')
        predicted_sources = util.collapse(np.stack(predicted_sources, axis=0))
        
        mean_localization_error = [eval_mean_localization_error(true_source, predicted_source, pos, argsorted_distance_matrix=argsorted_distance_matrix) for true_source, predicted_source in zip(true_sources, predicted_sources)]
        # auc = [eval_auc(true_source, predicted_source, pos, epsilon=0.25, n_redraw=25) for true_source, predicted_source in zip(true_sources, predicted_sources)]
        nmse = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in zip(true_sources, predicted_sources)]
        mse = [eval_mse(true_source, predicted_source) for true_source, predicted_source in zip(true_sources, predicted_sources)]
        
        mean_localization_errors[model_name].append(np.nanmedian(mean_localization_error))
        # aucs[model_name].append(np.nanmedian(auc))
        nmses[model_name].append(np.nanmedian(nmse))
        mses[model_name].append(np.nanmedian(mse))
    
%matplotlib qt
metrics = [mean_localization_errors, nmses, mses]
metric_names = ['mean_localization_error', 'nmses', 'mses']
x = (tmax-new_tmins)
for metric, metric_name in zip(metrics, metric_names):
    plt.figure()
    for key, val in metric.items():
        plt.plot(x, val, label=key)
    plt.legend()
    plt.title(metric_name)
    plt.xlabel("Duration of Trial for prediction")

# Calculate Evaluation Metrics

## Load Metrics

In [55]:
with open(f'results\\metrics_{len(true_sources)}_{sim_test.source_data[0].data.shape[1]}points_standard.pkl', 'rb') as f:
    [metrics, choice] = pkl.load(f)

# with open(f'simulations\\sim_test_5000_10points.pkl', 'rb') as f:
#     sim_lstm_test = pkl.load(f)


# Plot Evaluation Metrics

## Boxplot Overview 

In [194]:
import seaborn as sns; sns.set(style='whitegrid', font_scale=1.2, font='helvetica')
%matplotlib qt
classic_names = ['eLORETA', 'MNE', 'Beamformer']
for n in classic_names:
    if not n in model_names:
        model_names.append(n)
model_names
xticks = dict(ticks=np.arange(len(model_names)), labels=model_names)
plot = sns.boxplot  # violinplot
variable_keys = metrics.keys()
names = [ 'Mean Localization Errors',  'Far area under the curve', 'Close area under the curve', 'Combined Area under the curve', 'Normalized Mean Squared Errors', 'Mean Squared Errors']
plt.figure(figsize=(14, 8))
start = int((np.ceil(np.sqrt(len(names)))*100) + (np.ceil(np.sqrt(5))*10) + 1)
end = int(start + len(names))
subplot_nums = np.arange(start, end+1)
for variable_key, name, num in zip(variable_keys, names, subplot_nums):
    print(variable_key, name, num)
    plt.subplot(num)
    plot(data=metrics[variable_key])
    plt.title(name)
    plt.xticks(**xticks)
plt.tight_layout(pad=2)


mean_localization_errors Mean Localization Errors 331
aucs_far Far area under the curve 332
aucs_close Close area under the curve 333
aucs_combined Combined Area under the curve 334
nmses Normalized Mean Squared Errors 335
mses Mean Squared Errors 336


## Quadratic Scatter

In [101]:
sns.set(style='whitegrid')

plt.figure(figsize=(12, 10))
start = int((np.ceil(np.sqrt(len(names)))*100) + (np.ceil(np.sqrt(5))*10) + 1)
end = int(start + len(names))
subplot_nums = np.arange(start, end+1)
inverse_idc = [0,1]
for variable_key, name, num in zip(variable_keys, names, subplot_nums):
    var = np.array(metrics[variable_key])[inverse_idc]
    var = var[:, ~np.isnan(var).any(axis=0)]
    if np.any(np.isnan(var)):
        print("still nans")

    plt.subplot(num)
    plt.scatter(var[0], var[1], edgecolors='k', s=2)
    # origin
    lo_lim = np.nanmin(var)
    hi_lim = np.nanmax(var)
    plt.plot([lo_lim, hi_lim], [lo_lim, hi_lim], '--r')

    prop_higher = np.sum(var[1] > var[0]) / len(var[0])
    cohens_d = (np.nanmean(var[0]) - np.nanmean(var[1])) / np.mean([np.nanstd(var[0]), np.nanstd(var[1])])
    median_diff = np.abs(np.nanmedian(var[0]-var[1]))
    title = f'{name} (higher in {100*prop_higher:.1f} %)\nmedian_difference: {median_diff}\ncohens d: {abs(cohens_d):.2f}'
    plt.title(title)
    plt.xlabel(model_names[inverse_idc[0]])
    plt.ylabel(model_names[inverse_idc[1]])
    
    # pad
    lo_lim -= abs(hi_lim-lo_lim)*0.05
    hi_lim += abs(hi_lim-lo_lim)*0.05
    
    # plot properties
    plt.xlim(lo_lim, hi_lim)
    plt.ylim(lo_lim, hi_lim)
    plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()

## Dependence on anything

## Dependence on Noise

In [195]:
sim_test.simulation_info.columns

Index(['number_of_sources', 'positions', 'extents', 'amplitudes', 'shapes',
       'target_snr', 'betas', 'duration_of_trials'],
      dtype='object')

In [200]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = list(np.concatenate([[i]*src.data.shape[1] for i, src in enumerate(sim_test.source_data)]))
target_column = 'target_snr'
binning = True
n_bins = 4

params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
if binning:
    bins = np.linspace(df[target_column].min(), df[target_column].max()*1.01, num=n_bins)
    bin_labels = [str(int(round(bins[i]))) + ' - ' + str(int(round(bins[i+1]))) for i in range(len(bins)-1)]
    df['duration_bins'] = np.digitize(df[target_column].values, bins=bins)
    target_column = 'duration_bins'
else:
    bins = list(set(df[target_column].values))
    bins[-1] *= 1.01
    bin_labels = [str(bins[i]) for i in range(len(bins))]

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name.replace(' ', '_') + '_' + metric_name
        df[col_name] = metric[i]

print(df.columns)
dep_var_regex = target_column
dep_var_label = target_column.replace('_', ' ').title()
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]',  'AUC Combined [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

Index(['number_of_sources', 'positions', 'extents', 'amplitudes', 'shapes',
       'target_snr', 'betas', 'duration_of_trials', 'duration_bins',
       'LSTM_Standard_mean_localization_errors', 'LSTM_Standard_aucs_far',
       'LSTM_Standard_aucs_close', 'LSTM_Standard_aucs_combined',
       'LSTM_Standard_nmses', 'LSTM_Standard_mses',
       'LSTM_Noise_mean_localization_errors', 'LSTM_Noise_aucs_far',
       'LSTM_Noise_aucs_close', 'LSTM_Noise_aucs_combined', 'LSTM_Noise_nmses',
       'LSTM_Noise_mses', 'Dense_Standard_mean_localization_errors',
       'Dense_Standard_aucs_far', 'Dense_Standard_aucs_close',
       'Dense_Standard_aucs_combined', 'Dense_Standard_nmses',
       'Dense_Standard_mses', 'Dense_Noise_mean_localization_errors',
       'Dense_Noise_aucs_far', 'Dense_Noise_aucs_close',
       'Dense_Noise_aucs_combined', 'Dense_Noise_nmses', 'Dense_Noise_mses',
       'ConvDip_Standard_mean_localization_errors',
       'ConvDip_Standard_aucs_far', 'ConvDip_Standard_aucs_clo

In [94]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_test.n_samples), np.mean([src.data.shape[1] for src in sim_test.source_data]))
snr_bins = (2, 4, 6, 8, 10, 20)
# snr_bins = (0., 10, 20)

snr_bin_labels = [str(snr_bins[i]) + ' - ' + str(snr_bins[i+1]) for i in range(len(snr_bins)-1)]
params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
df['target_snr_bins'] = np.digitize(df.target_snr, bins=snr_bins)
# df['avg_eccentricity'] = [np.median((df.positions.values[i]**2).sum(axis=1)**(1/2)) for i in range(df.shape[0])]
# df['avg_extent'] = [np.mean(df.extents.values[i]) for i in range(df.shape[0])]
# df['avg_beta'] = [np.mean(df.betas.values[i]) for i in range(df.shape[0])]

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex='snr_'), df.filter(regex=metric_name)), axis=1).melt('target_snr_bins', var_name='cols', value_name='vals')
    g = sns.catplot(x='target_snr_bins', y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=snr_bin_labels, ylabel=metric_name_nice, xlabel='Signal to Noise Ratio (SNR)')
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

Index(['number_of_sources', 'positions', 'extents', 'amplitudes', 'shapes',
       'target_snr', 'betas', 'target_snr_bins',
       'LSTM Standard_mean_localization_errors', 'LSTM Standard_aucs_far',
       'LSTM Standard_aucs_close', 'LSTM Standard_aucs_combined',
       'LSTM Standard_nmses', 'LSTM Standard_mses',
       'LSTM Noise_mean_localization_errors', 'LSTM Noise_aucs_far',
       'LSTM Noise_aucs_close', 'LSTM Noise_aucs_combined', 'LSTM Noise_nmses',
       'LSTM Noise_mses', 'Dense Standard_mean_localization_errors',
       'Dense Standard_aucs_far', 'Dense Standard_aucs_close',
       'Dense Standard_aucs_combined', 'Dense Standard_nmses',
       'Dense Standard_mses', 'Dense Noise_mean_localization_errors',
       'Dense Noise_aucs_far', 'Dense Noise_aucs_close',
       'Dense Noise_aucs_combined', 'Dense Noise_nmses', 'Dense Noise_mses',
       'ConvDip Standard_mean_localization_errors',
       'ConvDip Standard_aucs_far', 'ConvDip Standard_aucs_close',
       'ConvDip

## Dependence on eccentricity

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_test.n_samples), np.mean([src.data.shape[1] for src in sim_test.source_data]))



params = list(sim_lstm_test.simulation_info.columns)
series = [sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param] for param in params]
df = pd.concat(series, axis=1)
df['avg_eccentricity'] = [np.median((df.positions.values[i]**2).sum(axis=1)**(1/2)) for i in range(df.shape[0])]
ecc_bins = np.linspace(int(df['avg_eccentricity'].values.min()), np.ceil(df['avg_eccentricity'].values.max()).astype(int), num=5)
ecc_bin_labels = [str(ecc_bins[i]) + ' - ' + str(ecc_bins[i+1]) for i in range(len(ecc_bins)-1)]
df['avg_eccentricity_bins'] = np.digitize(df['avg_eccentricity'].values, bins=ecc_bins)

df['avg_extent'] = [np.mean(df.extents.values[i]) for i in range(df.shape[0])]
df['avg_beta'] = [np.mean(df.betas.values[i]) for i in range(df.shape[0])]

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
dep_var_regex = 'avg_eccentricity_bins'  # 'target_snr_bins'
dep_var_label = 'Average eccentricity'  # 'Signal to Noise Ratio (SNR)'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=ecc_bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Dependence on No. of sources

In [181]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = list(np.concatenate([[i]*src.data.shape[1] for i, src in enumerate(sim_test.source_data)]))

# nsrc_bins = (1, 2, 4, 8, 16, 20)
nsrc_bins = (1, 2, 4, 8, 10)

nsrc_bin_labels = [str(nsrc_bins[i]) + ' - ' + str(nsrc_bins[i+1]) for i in range(len(nsrc_bins)-1)]

params = list(sim_test.simulation_info.columns)
series = [sim_test.simulation_info.iloc[stretched_indices][param] for param in params]
df = pd.concat(series, axis=1)
df['number_of_sources_bins'] = np.digitize(df['number_of_sources'].values, bins=nsrc_bins)

for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name
        df[col_name] = metric[i]

print(df.columns)
dep_var_regex = 'number_of_sources_bins'  # 'target_snr_bins'
dep_var_label = 'Number of True Sources'  # 'Signal to Noise Ratio (SNR)'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex=metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=nsrc_bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()


Index(['number_of_sources', 'positions', 'extents', 'amplitudes', 'shapes',
       'target_snr', 'betas', 'duration_of_trials', 'number_of_sources_bins',
       'LSTM Standard_mean_localization_errors', 'LSTM Standard_aucs_far',
       'LSTM Standard_aucs_close', 'LSTM Standard_aucs_combined',
       'LSTM Standard_nmses', 'LSTM Standard_mses',
       'LSTM Noise_mean_localization_errors', 'LSTM Noise_aucs_far',
       'LSTM Noise_aucs_close', 'LSTM Noise_aucs_combined', 'LSTM Noise_nmses',
       'LSTM Noise_mses', 'Dense Standard_mean_localization_errors',
       'Dense Standard_aucs_far', 'Dense Standard_aucs_close',
       'Dense Standard_aucs_combined', 'Dense Standard_nmses',
       'Dense Standard_mses', 'Dense Noise_mean_localization_errors',
       'Dense Noise_aucs_far', 'Dense Noise_aucs_close',
       'Dense Noise_aucs_combined', 'Dense Noise_nmses', 'Dense Noise_mses',
       'ConvDip Standard_mean_localization_errors',
       'ConvDip Standard_aucs_far', 'ConvDip Standard

ValueError: The number of FixedLocator locations (1), usually from a call to set_ticks, does not match the number of ticklabels (4).

## Dependence on Beta

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_lstm_test.n_samples), sim_lstm_test.source_data[0].data.shape[1])
snr_bins = (0.5, 2, 4, 6, 8, 10)
snr_bin_labels = [str(snr_bins[i]) + ' - ' + str(snr_bins[i+1]) for i in range(len(snr_bins)-1)]

beta_bins = (0.5, 0.75, 1, 1.25, 1.5)
beta_bin_labels = [str(beta_bins[i]) + ' - ' + str(beta_bins[i+1]) for i in range(len(beta_bins)-1)]


params = list(sim_lstm_test.simulation_info.columns)
series = [sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param] for param in params]
df = pd.concat(series, axis=1)
df['target_snr_bins'] = np.digitize(df.target_snr, bins=snr_bins)

df['avg_beta'] = [np.mean(df.betas.values[i]) for i in range(df.shape[0])]
df['avg_beta_bins']  = np.digitize(df.avg_beta, bins=beta_bins)


for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
regex = 'beta_'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=regex), df.filter(regex=metric_name)), axis=1).melt('avg_beta_bins', var_name='cols', value_name='vals')
    g = sns.catplot(x='avg_beta_bins', y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=beta_bin_labels, ylabel=metric_name_nice, xlabel='Beta Exponent')
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Dependence on Extent

In [ ]:
import pandas as pd
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
stretched_indices = np.repeat(np.arange(sim_lstm_test.n_samples), sim_lstm_test.source_data[0].data.shape[1])

extent_bins = np.linspace(20, 40, 5)
extent_bin_labels = [str(int(round(extent_bins[i]))) + ' - ' + str(int(round(extent_bins[i+1]))) for i in range(len(extent_bins)-1)]

params = list(sim_lstm_test.simulation_info.columns)
series = [sim_lstm_test.simulation_info.iloc[stretched_indices[choice]][param] for param in params]
df = pd.concat(series, axis=1)

df['avg_extent'] = [np.mean(df.extents.values[i]) for i in range(df.shape[0])]
df['avg_extent_bins']  = np.digitize(df.avg_extent, bins=extent_bins)


for i, model_name in enumerate(model_names):
    for metric_name, metric in metrics.items():
        col_name = model_name + '_' + metric_name

        df[col_name] = metric[i]

print(df.columns)
regex = 'extent_'
metric_names_nice = ['Mean Localization Error [mm]', 'AUC Far [%]', 'AUC Close [%]', 'Normalized Mean Squared Error', 'Mean Squared Error']
for metric_name, metric_name_nice in zip(metrics.keys(), metric_names_nice):
    df_temp = pd.concat((df.filter(regex=regex), df.filter(regex=metric_name)), axis=1).melt('avg_extent_bins', var_name='cols', value_name='vals')
    g = sns.catplot(x='avg_extent_bins', y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=extent_bin_labels, ylabel=metric_name_nice, xlabel='Source extent [mm]')
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Save all figures

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

def multipage(filename, figs=None, dpi=300, png=False):
    ''' Saves all open (or list of) figures to filename.pdf with dpi''' 
    pp = PdfPages(filename)
    path = os.path.dirname(filename)
    fn = os.path.basename(filename)[:-4]

    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for i, fig in enumerate(figs):
        print(f'saving fig {fig}\n')
        fig.savefig(pp, format='pdf', dpi=dpi)
        if png:
            fig.savefig(f'{path}\\{i}_{fn}.png', dpi=600)
    pp.close()

multipage(f'figures\\figs_5000_10samples\\figs_5000_10samples.pdf', png=True)

In [ ]:
import seaborn as sns
%matplotlib qt
model_names[0] = 'Dense'

data = {model_name: nmse for model_name, nmse in zip(model_names, nmses)}
plt.figure()
sns.kdeplot(data=data, multiple='stack')
plt.title('Normalized Mean Squared Errors')

plt.figure()
data = {model_name: mse for model_name, mse in zip(model_names, mses)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Mean Squared Errors')


plt.figure()
data = {model_name: auc for model_name, auc in zip(model_names, aucs_far)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Far area under the curve')


plt.figure()
data = {model_name: auc for model_name, auc in zip(model_names, aucs_close)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Close area under the curve')

plt.figure()
data = {model_name: mle for model_name, mle in zip(model_names, mean_localization_errors)}
sns.kdeplot(data=data, multiple='stack')
plt.title('Close area under the curve')
plt.title('Mean Localization Errors')

plt.tight_layout()

# Performance per Difficulty Evaluation

In [ ]:
metrics = [nmses, mean_localization_errors, aucs_far, aucs_close]
metric_names = ['nmses', 'mean_localization_errors', 'aucs_far', 'aucs_close']

covariates = ['target_snr', 'number_of_sources']

n_samples = int(np.array(nmses).shape[1] / 20)
for covariate in covariates:
    sim_params = sim_lstm_test.simulation_info[covariate].values[:n_samples]
    sim_params = np.repeat(sim_params, 20)
    for metric, metric_name in zip(metrics, metric_names):
        plt.figure()
        for i, model_name in enumerate(model_names):
            plt.scatter(sim_params, metric[i], label=model_name, s=.7)
        plt.title(f'{covariate}, {metric_name}')
        plt.legend()


## Speed test

In [ ]:
import time

def new_sim_params(sr=100, packages_per_second=20):
    package_size = int( round( sr / packages_per_second  ) )
    package_interval = package_size/sr

    n_chan = len(sim_lstm_test.eeg_data.ch_names)
    data_package = np.random.randn(n_chan, package_size)

    sim_data_package = Simulation(fwd, info, settings=dict(duration_of_trial=0.01*package_size)).simulate(1)
    print(f'performing predictions {packages_per_second} times per second')

    return sim_data_package, package_interval

packages_per_second = 50
sim_data_package, package_interval = new_sim_params(packages_per_second=packages_per_second)

while True:
    start = time.time()
    # stc = net_dense.predict(sim_data_package)
    stc = net_lstm.predict(sim_data_package)

    stop = time.time()
    diff = stop-start
    if stop-start > package_interval:
        print(f"took longer than expected: {diff} (instead of {package_interval})")
        print(f'decreasing package interval by one')
        packages_per_second -= 1
        sim_data_package, package_interval = new_sim_params(packages_per_second=packages_per_second)
        print(f'packages_per_second={packages_per_second}\n')
        continue
    time.sleep(package_interval-diff)

